Library

In [1]:
import pandas as pd
import os
import re

In [2]:
API_KEY = "sk-ms7SU43E34tS9UJks5RD2KM3m1JumOR2pM73Dk95VzKjM6TZ"

API_KEY = API_KEY or os.getenv("H2O_GPT_E_API_KEY")

if not API_KEY:
    raise ValueError("Please configure h2ogpte API key")

REMOTE_ADDRESS = "https://h2ogpte.genai.h2o.ai"

from h2ogpte import H2OGPTE

client = H2OGPTE(address=REMOTE_ADDRESS, api_key=API_KEY)

Common issue extration by topic

In [17]:
tor = pd.read_csv('../Data/LDA_Reviews.csv.csv')

In [4]:
def get_review_issue(topic_name):
    issue1 = tor.loc[tor['Topic'] == topic_name]
    return issue1

Convert all the reviews into a long list of string

In [5]:
#first convert the extracted/filtered topic reviews extracted into strings
def convert_str(topic_review):
    compiled_review = topic_review['review'].to_string()

    comp_rev = ""
    for row in compiled_review:
        comp_rev += row
    return(comp_rev) #returns a long string

Extract the issues using gpt

In [6]:
#extract common issues and its corresponding review number, but all in one string
def common_issues(review):
    extract = client.extract_data(
        text_context_list= [review],
        #pre_prompt_extract="Pay attention and look at all people. Your job is to collect their names.\n",
        prompt_extract="Extract all the common issues faced by the application according to the reviews, as well as their corresponding index after the identified issues. Do not return positive. Strictly return in the following format, issues : indexes without additional information"
    )
    # List of LLM answers per text input
    sol = []
    for extract_list_item in extract.content:
        for s in extract_list_item.split("\n"):
            s = re.sub('\W+',' ', s)
            sol.append(s)
    return sol

<>:12: SyntaxWarning: invalid escape sequence '\W'
<>:12: SyntaxWarning: invalid escape sequence '\W'
/var/folders/1l/pycwvg157rv_sn0s68n1_y8c0000gn/T/ipykernel_85121/4238437813.py:12: SyntaxWarning: invalid escape sequence '\W'
  s = re.sub('\W+',' ', s)


Cleaned up the output obtained from common_issues function. Better formatting such that each review is assigned with their respective issue

In [7]:
#format it so that they are in dic form, for better df manipulation
def extract_review(issue_ind):
    result_dict = {}
    for item in issue_ind:
        # Use regex to extract the string part without numbers
        string = re.sub(r'\d+', '', item)  # Replace all digits with an empty string
        string = string.strip()  # Remove leading and trailing whitespaces
        
        numbers = [int(num) for num in item.split() if num.isdigit()]  # Extract all numbers in the string
        if string in result_dict:
            result_dict[string].extend(numbers)
        else:
            result_dict[string] = numbers
    
    return result_dict

# Overall function for identifying issues within each topic

In [15]:
#combine the reviews on issues tgt. Damn just combine everything above TT
def issue_df(topic):
    issue1 = tor.loc[tor['Topic'] == topic]
    review_str = convert_str(issue1)
    review_lst = common_issues(review_str)
    dic = extract_review(review_lst)
    
    # Get all indices covered by the dictionary values
    covered_indices = set(index for indices in dic.values() for index in indices)

    # Get all indices in the tof dataset
    all_indices = set(issue1['index'])

    # Get the remaining indices
    remaining_indices = all_indices - covered_indices

    # Create an empty list to hold individual DataFrames
    dfs = []

    # Iterate over the dictionary items
    for key, indices in dic.items():
    # Create a DataFrame for each key-value pair
        indices_with_reviews = [index for index in indices if index in issue1.index]
        df = pd.DataFrame({'key': [key]*len(indices_with_reviews), 'review': [issue1.loc[i, 'review'] for i in indices_with_reviews]})
        dfs.append(df)

# Add remaining indices under "Others"
    if remaining_indices:
        others_indices_with_reviews = [index for index in remaining_indices if index in issue1.index]
        others_df = pd.DataFrame({'key': ['Others']*len(others_indices_with_reviews), 'review': [issue1.loc[i, 'review'] for i in others_indices_with_reviews]})
        dfs.append(others_df)

    # Concatenate all DataFrames together
    result_df = pd.concat(dfs, ignore_index=True)

    return(result_df)


To use the function, simply type in the topic which you want to extract the specific issues from

In [18]:
the_df = issue_df("Others")
the_df

,key,review
0,Difficulty logging in,Occasionally getting unable to login error
1,Difficulty logging in,I not be able to log in GXS bank today after I...
2,Difficulty logging in,I have been trying for few hours to signed up ...
3,Difficulty logging in,Was rejected without details
4,Difficulty logging in,Unable to open the app once Ive installed
5,Unable to open the app,Same cannot open the apps now What happened
6,Unable to open the app,Unable to open the app once Ive installed
7,Unable to open the app,Always freezing and crashes
8,Unable to open the app,One of the worst app I ever installed Such a p...
9,Freezing and crashing,Always freezing and crashes


To save the dataframe from above, rename the name accordingly and run the code below

In [19]:
#save df
the_df.to_csv('../Data/Others.csv')